In [ ]:
from qiskit_ibm_provider import IBMProvider
provider = IBMProvider(token='MY_API_KEY')


# https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq

for backend in provider.backends(filters=lambda x: not x.configuration().simulator and x.status().operational==True):
    
    name = backend.name
    num_q = backend.num_qubits
    properties = backend.properties().to_dict()['qubits']
    avr_t1 = np.mean([properties[nq][0]['value'] for nq in range(num_q)])  # Average T1 time
    avr_t2 = np.mean([properties[nq][1]['value'] for nq in range(num_q)])  # Average T2 time
    avr_ro = np.mean([properties[nq][4]['value'] for nq in range(num_q)])  # Average readout. Mean of prep0_meas1 and prep1_meas0
    pending_jobs = backend.status().pending_jobs
    
    one_qb_err = []
    two_qb_err = []
    for gate_i in backend.properties().gates:

        if "rz" in gate_i.name:                                            # Ignore Rz gates as it is a classical phase shift with error of 0
            continue
        if "reset" in gate_i.name:                                         # Ignore resets for now since they're given in units of ns
            continue
        if len(gate_i.qubits) == 1:
            one_qb_err.append(gate_i.parameters[0].value)
        else:
            two_qb_err.append(gate_i.parameters[0].value)
            
    avr_1err = np.mean(one_qb_err)                                          # Average error rates for 1 qubit gates
    avr_2err = np.mean(two_qb_err)                                          # Average error rates for 2 qubit gates
    
    
    print(f"Name: {name:<16}, Avr T1 (μs): {avr_t1:<7.3f}, Avr T2 (μs): {avr_t2:<7.3f}, Avr 1qb err: {avr_1err:<7.6f}, Avr 2qb err: {avr_2err:<7.5f}",
          f"Avr RO: {avr_ro:<6.4f},", 
          # f"Jobs in Q: {pending_jobs:<3d}"                                # Seems to be broken for some reason.
         )
    
